<a href="https://colab.research.google.com/github/vigneswar96/vigneswar96/blob/main/Sentiment_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sentiment Classifcation 

## Dataset 
- Dataset of 50,000 movie reviews from IMDB, labeled by sentiment positive (1) or negative (0)
- Reviews have been preprocessed, and each review is encoded as a sequence of word indexes (integers).
- For convenience, words are indexed by overall frequency in the dataset, so that for instance the integer "3" encodes the 3rd most frequent word in the data. This allows for quick filtering operations such as: "only consider the top 10,000 most common words, but eliminate the top 20 most common words".
- As a convention, "0" does not stand for a specific word, but instead is used to encode any unknown word.

Command to import data
- `from tensorflow.keras.datasets import imdb`

In [1]:
from tensorflow.keras.datasets import imdb 
(X_train, y_train), (X_test,y_test) = imdb.load_data(num_words = 10000)


17464789/17464789 [==============================] - 2s 0us/step


### Pad each sentence to be of same length 

- Take maximum sequence length as 300

In [2]:
from tensorflow.keras.preprocessing.sequence import pad_sequences 

X_train = pad_sequences(X_train, maxlen = 300)
X_test = pad_sequences (X_test, maxlen = 300)  

## Print Shape of features and labels 

- Number of review, number of words in each review 

In [3]:
X_train.shape

(25000, 300)

In [4]:
X_test.shape

(25000, 300)

### Number of labels 

In [5]:
y_train.shape

(25000,)

In [6]:
y_test.shape

(25000,)

In [7]:
X_train

array([[   0,    0,    0, ...,   19,  178,   32],
       [   0,    0,    0, ...,   16,  145,   95],
       [   0,    0,    0, ...,    7,  129,  113],
       ...,
       [   0,    0,    0, ...,    4, 3586,    2],
       [   0,    0,    0, ...,   12,    9,   23],
       [   0,    0,    0, ...,  204,  131,    9]], dtype=int32)

In [8]:
y_train[1]

0

Decode the feature value to get original sentence 

### First, retrieve a dictionary that contains mapping of words to their index in the IMDB dataset

In [9]:
word_index_dict = imdb.get_word_index()

1641221/1641221 [==============================] - 1s 1us/step


### Now I am gonna use the original words from the encodings, for a particular sentence 

In [10]:
for encoding in X_train[0]:
  for key, value in word_index_dict.items():
    if encoding == value:
      print(key, end= " ")
      

the as you with out themselves powerful lets loves their becomes reaching had journalist of lot from anyone to have after out atmosphere never more room and it so heart shows to years of every never going and help moments or of every chest visual movie except her was several of enough more with is now current film as you of mine potentially unfortunately of you than him that with out themselves her get for was camp of you movie sometimes movie that with scary but and to story wonderful that in seeing in character to of 70s musicians with heart had shadows they of here that with her serious to have does when from why what have critics they is you that isn't one will very to as itself with other and in of seen over landed for anyone of and br show's to whether from than out themselves history he name half some br of and odd was two most of mean for 1 any an boat she he should is thought frog but of script you not while history he heart to real at barrel but when from one bit then have tw

### I know it seems to be pretty weird to read from the narrow line 

### Now lets try to get the snetiment for the above sentence 
- Positive(1)
- Negative(0)

In [11]:
y_train[0]

1

### Lets define the Model. Come let's go ✈ 

### 
- Define Model 
- Embedding layer 
   -  Embedding layer turns postive integers into dense vectors of fixed size
   -  tensorflow.keras embedding layer doesn't require us to onehot encode our words, instead we have to give each unique word a unique integer number as an id. For the imdb dataset we've loaded this has already been done, but if this wan't the case we could use sklearn LabelEncoder 
    -  Size of the vocabulory will be 10000
    -  Give dimension of the dense embedding as 100
    -  Length of input sequences should be 300 
    -  Add LSTM layer 
       -  Pass value in return_sequences as True 
    - Add a Time distributed layer with 100 Dense neurons 
    - Add Flatten layer 
    - Add Dense layer 

In [12]:
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Embedding, LSTM , Flatten, Dense, TimeDistributed 

model = Sequential()
model.add(Embedding(input_dim=10000, output_dim = 100, input_length = 300))
model.add(LSTM(64, activation = 'relu', return_sequences=True))

model.add(TimeDistributed(Dense(100)))

model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dense(512, activation= 'relu'))
model.add(Dense(256, activation = 'relu'))
model.add(Dense(1, activation='sigmoid'))



## That's enough for today 

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 300, 100)          1000000   
                                                                 
 lstm (LSTM)                 (None, 300, 64)           42240     
                                                                 
 time_distributed (TimeDistr  (None, 300, 100)         6500      
 ibuted)                                                         
                                                                 
 flatten (Flatten)           (None, 30000)             0         
                                                                 
 dense_1 (Dense)             (None, 1024)              30721024  
                                                                 
 dense_2 (Dense)             (None, 512)               524800    
                                                        

## Oops that's a lot of parameters 😵

### Anyways lets compile 🖖

### Lets Compile the model now 
- Using Adam as Optimizer 
- Using Binary Crossentropy as loss 
- Please dont ask me what Binary crossentropy is 😛
- Use Accuracy as metrics  

In [14]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics= ['accuracy'])

In [15]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5, batch_size=128)

Epoch 1/5
196/196 [==============================] - 166s 809ms/step - loss: 0.3917 - accuracy: 0.7987 - val_loss: 0.2977 - val_accuracy: 0.8740
Epoch 2/5
196/196 [==============================] - 140s 717ms/step - loss: 0.1906 - accuracy: 0.9274 - val_loss: 0.3169 - val_accuracy: 0.8767
Epoch 3/5
196/196 [==============================] - 137s 700ms/step - loss: 0.1039 - accuracy: 0.9625 - val_loss: 0.4565 - val_accuracy: 0.8637
Epoch 4/5
196/196 [==============================] - 136s 694ms/step - loss: 0.0431 - accuracy: 0.9844 - val_loss: 0.5414 - val_accuracy: 0.8605
Epoch 5/5
196/196 [==============================] - 131s 670ms/step - loss: 0.0256 - accuracy: 0.9906 - val_loss: 0.8273 - val_accuracy: 0.8564


In [16]:
score = model.evaluate(X_test, y_test)
print("Loss: {}, Accuracy:{}".format(score[0], score[1]))

782/782 [==============================] - 38s 49ms/step - loss: 0.8273 - accuracy: 0.8564
Loss: 0.8273101449012756, Accuracy:0.8563600182533264


In [17]:
model.predict(X_test[0].reshape((1, 300)))

1/1 [==============================] - 0s 267ms/step


array([[0.00016712]], dtype=float32)